In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvid

In [21]:
import pandas as pd

base_path = "/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail"

# Load datasets
train_df = pd.read_csv(f"{base_path}/train.csv")
test_df = pd.read_csv(f"{base_path}/test.csv")
val_df = pd.read_csv(f"{base_path}/validation.csv")


In [22]:
# Preview
train_df.head(4)

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...


In [23]:
test_df.head(4)

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...


In [24]:
train_df.drop(columns = ['id'],inplace = True)
test_df.drop(columns = ['id'],inplace = True)
val_df.drop(columns = ['id'],inplace = True)

In [25]:
train_df['article_length'] = [len(x.split()) for x in train_df.article]
train_df['highlights_length'] = [len(x.split()) for x in train_df.highlights]
train_df.head(10)

,article,highlights,article_length,highlights_length
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...",198,37
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...,392,38
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t...",808,69
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...,531,53
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...,580,67
5,He's been accused of making many a fashion fau...,Prime Minister and his family are enjoying an ...,334,42
6,By . Daily Mail Reporter . PUBLISHED: . 01:15 ...,NBA star calls for black and Hispanic communit...,742,23
7,By . Daily Mail Reporter . This is the moment ...,London Midland service had been pulling into T...,621,54
8,There are a number of job descriptions waiting...,Tony Pulis believes Saido Berahino should look...,609,59
9,"Canberra, Australia (CNN) -- At first glance, ...",Black box data from Flight 370 could be analyz...,888,56


In [26]:
train_df.drop(columns = ['article_length','highlights_length'],inplace = True)

In [29]:
from datasets import Dataset

# Drop any missing data just in case
train_df = train_df[['article', 'highlights']].dropna().sample(n=100000, random_state=42).reset_index(drop=True)

test_df = test_df[['article', 'highlights']].dropna()
val_df = val_df[['article', 'highlights']].dropna()

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
val_dataset = Dataset.from_pandas(val_df)


In [31]:
from transformers import BartTokenizer, BartForConditionalGeneration

model_name = "facebook/bart-large-xsum"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

In [32]:
max_input_length = 1024
max_target_length = 128

def tokenize_function(example):
    model_inputs = tokenizer(
        example["article"],
        max_length=max_input_length,
        padding="max_length",
        truncation=True,
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["highlights"],
            max_length=max_target_length,
            padding="max_length",
            truncation=True,
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["article", "highlights"])
tokenized_test = test_dataset.map(tokenize_function, batched=True, remove_columns=["article", "highlights"])
tokenized_val = val_dataset.map(tokenize_function, batched=True, remove_columns=["article", "highlights"])


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments
import os
os.environ["WANDB_DISABLED"] = "true"

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=6,  
    predict_with_generate=True,
    fp16=True,  
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [51]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [52]:
from transformers import Seq2SeqTrainer


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/tmp/ipykernel_35/2867047103.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [53]:
trainer.train_dataset = tokenized_train.select(range(3000))
trainer.eval_dataset = tokenized_val.select(range(300))
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.419600,1.109542
2,0.270700,1.268932
3,0.198700,1.343234
4,0.154900,1.520363
5,0.103300,1.592892
6,0.073700,1.649224


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

TrainOutput(global_step=4500, training_loss=0.2075753724839952, metrics={'train_runtime': 7716.5747, 'train_samples_per_second': 2.333, 'train_steps_per_second': 0.583, 'total_flos': 3.9007882838016e+16, 'train_loss': 0.2075753724839952, 'epoch': 6.0})

In [54]:
!pip install evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=eb8918f3c768844899bfb45b6bb52cb13e2b33c3f5da907219f43d131e0c0ddf
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import evaluate
import numpy as np
from tqdm import tqdm 

# Load ROUGE metric
rouge = evaluate.load("rouge")

# Take 50 test samples
sampled_test = test_df[['article', 'highlights']].dropna().iloc[:50].reset_index(drop=True)

preds = []
refs = []

# Generate summaries
for i in tqdm(range(50), desc="Generating Summaries"):
    article = sampled_test["article"][i]
    reference = sampled_test["highlights"][i]

    inputs = tokenizer(
        article,
        return_tensors="pt",
        max_length=1024,
        truncation=True,
        padding="max_length"
    ).to(model.device)

    summary_ids = model.generate(
        inputs["input_ids"],
        num_beams=4,
        max_length=128,
        early_stopping=True
    )

    decoded_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    preds.append(decoded_summary)
    refs.append(reference)

# Compute ROUGE scores
results = rouge.compute(predictions=preds, references=refs)

# Print the scores
print("\n ROUGE Evaluation Results on 50 Test Samples:\n")
for metric, score in results.items():
    print(f"{metric.upper():<10}: {score:.4f}")


Generating Summaries: 100%|██████████| 50/50 [01:17<00:00,  1.55s/it]



 ROUGE Evaluation Results on 50 Test Samples:

ROUGE1    : 0.4045
ROUGE2    : 0.1790
ROUGEL    : 0.2750
ROUGELSUM : 0.3747


In [59]:
model_path = "my_model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)


('my_model/tokenizer_config.json',
 'my_model/special_tokens_map.json',
 'my_model/vocab.json',
 'my_model/merges.txt',
 'my_model/added_tokens.json')

In [60]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [67]:
for i in range(5):
    print("original_news: ",test_df['article'].iloc[i])
    summarizer = pipeline("summarization", model = model,tokenizer = tokenizer, max_length = 128)
    summary = summarizer(test_df['article'].iloc[i])
    print(summary[0])
    print()

Device set to use cuda:0


original_news:  Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee. 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fig

Device set to use cuda:0


{'summary_text': "U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans .\nMany economy seats on United Airlines have 30 inches of room, while some airlines offer as little as 28 inches .\nBritish Airways has a seat pitch of 31 inches, while easyJet has 29 inches ."}

original_news:  A drunk teenage boy had to be rescued by security after jumping into a lions' enclosure at a zoo in western India. Rahul Kumar, 17, clambered over the enclosure fence at the Kamla Nehru Zoological Park in Ahmedabad, and began running towards the animals, shouting he would 'kill them'. Mr Kumar explained afterwards that he was drunk and 'thought I'd stand a good chance' against the predators. Next level drunk: Intoxicated Rahul Kumar, 17, climbed into the lions' enclosure at a zoo in Ahmedabad and began running towards the anima

Device set to use cuda:0


{'summary_text': "Rahul Kumar, 17, jumped into lions' enclosure at the Kamla Nehru Zoological Park in Ahmedabad .\nHe ran towards the animals shouting 'Today I kill a lion or a lion kills me!'\nThe intoxicated teenager then fell into a moat as he tried to escape the enclosure .\nMr Kumar later explained: 'I don't really know why I did it. I was drunk and thought I'd stand a good chance'\nHe will be sent for psychiatric evaluation after the incident ."}

original_news:  Dougie Freedman is on the verge of agreeing a new two-year deal to remain at Nottingham Forest. Freedman has stabilised Forest since he replaced cult hero Stuart Pearce and the club's owners are pleased with the job he has done at the City Ground. Dougie Freedman is set to sign a new deal at Nottingham Forest . Freedman has impressed at the City Ground since replacing Stuart Pearce in February . They made an audacious attempt on the play-off places when Freedman replaced Pearce but have tailed off in recent weeks. That h

Device set to use cuda:0


{'summary_text': 'Dougie Freedman close to agreeing two-year contract extension at Nottingham Forest .\nThe Forest boss has stabilised the club since replacing Stuart Pearce in February .\nOwners are pleased with the job Freedman has done since taking over .'}

original_news:  Liverpool target Neto is also wanted by PSG and clubs in Spain as Brendan Rodgers faces stiff competition to land the Fiorentina goalkeeper, according to the Brazilian's agent Stefano Castagna. The Reds were linked with a move for the 25-year-old, whose contract expires in June, earlier in the season when Simon Mignolet was dropped from the side. A January move for Neto never materialised but the former Atletico Paranaense keeper looks certain to leave the Florence-based club in the summer. Neto rushes from his goal as Juan Iturbe bears down on him during Fiorentina's clash with Roma in March . Neto is wanted by a number of top European clubs including Liverpool and PSG, according to his agent . It had been repor

Device set to use cuda:0


{'summary_text': 'Novo is wanted by a number of top European clubs including Liverpool and PSG, according to his agent .\nLiverpool were linked with a move for the goalkeeper earlier in the season .\nPSG and Real Madrid are also interested in the 25-year-old goalkeeper .\nHe looks set to leave Fiorentina in the summer after agreeing a contract extension .'}

original_news:  Bruce Jenner will break his silence in a two-hour interview with Diane Sawyer later this month. The former Olympian and reality TV star, 65, will speak in a 'far-ranging' interview with Sawyer for a special edition of '20/20' on Friday April 24, ABC News announced on Monday. The interview comes amid growing speculation about the father-of-six's transition to a woman, and follows closely behind his involvement in a deadly car crash in California in February. And while the Kardashian women are known for enjoying center stage, they will not be stealing Bruce's spotlight because they will be in Armenia when the intervie